In [113]:
from morphcloud.api import MorphCloudClient
import os

In [114]:
client = MorphCloudClient()

In [115]:
# initialize base instance
snapshot = client.snapshots.create(
    vcpus=1,
    memory=4096,
    disk_size=8192
)

In [74]:
snapshot = (
    snapshot.setup("apt update -y")
    .setup("apt-get install -y python3-pip")
    .setup("curl -LsSf https://astral.sh/uv/install.sh | bash")
)

Effect function: _run_command_effect
args: ()   kwargs: {'command': 'apt update -y', 'background': False, 'get_pty': True}

🚀 Building new snapshot with digest 61eee48f351413037af7870e65c2a00aefc89b9506be66dde28e226badf5bfa4.

🔧 Running command (foreground): apt update -y

Output()

🎉 New snapshot created with digest 61eee48f351413037af7870e65c2a00aefc89b9506be66dde28e226badf5bfa4.

...

Effect function: _run_command_effect
args: ()   kwargs: {'command': 'curl -LsSf https://astral.sh/uv/install.sh | bash', 'background': False, 'get_pty':
True}

🚀 Building new snapshot with digest db9306addd0343592e29e7e4ef7c4554334d7cb6572c512d9b2283f69d86b14d.

🔧 Running command (foreground): curl -LsSf https://astral.sh/uv/install.sh | bash

Output()

🎉 New snapshot created with digest db9306addd0343592e29e7e4ef7c4554334d7cb6572c512d9b2283f69d86b14d.

# TMP: until we find out how to upload files to a snapshot with the setup api we will manually do it to an instance

In [116]:
base_instance = client.instances.start(snapshot_id=snapshot.id)

In [76]:
result = base_instance.exec(command="mkdir app")
# upload project to base instance
with base_instance.ssh() as ssh:
    sftp = ssh._client.open_sftp()

    sftp.put("/home/asusevski/morph/chess/config.yaml", "app/config.yaml")
    sftp.put("/home/asusevski/morph/chess/pyproject.toml", "app/pyproject.toml")
    sftp.put("/home/asusevski/morph/chess/requirements.txt", "app/requirements.txt")
    sftp.put("/home/asusevski/morph/chess/schemas.py", "app/schemas.py")
    sftp.put("/home/asusevski/morph/chess/chess_game.py", "app/chess_game.py")
    sftp.put("/home/asusevski/morph/chess/chess_engine.py", "app/chess_engine.py")
    sftp.put("/home/asusevski/morph/chess/agent.py", "app/agent.py")

# Remaining to setup: uv add pyproject.toml
- NOTE: can just keep the tmp instance i guess

In [77]:
# branch from instance with files transferred + uv + deps installed
base_snapshot, clones = base_instance.branch(count=3)
print(f"Snapshot created: {base_snapshot.id}. This is the base snapshot for all chess branches.")

Snapshot created: snapshot_sxocpfn8. This is the base snapshot for all chess branches.


In [91]:
[clone.id for clone in clones]

['morphvm_0lupadv4', 'morphvm_pwkuoroc', 'morphvm_fa6uh8o9']

In [78]:
NUM_MOVES = 5
game_ids = [1,2,3]
strategies = ["aggressive", "defensive", "balanced"]
#instance_startup = f"export OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY')} && cd app && mkdir chess_autosaves && uv add pyproject.toml && source .venv/bin/activate && python agent.py"
run_agent_cmd = f"export OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY')} && cd app && mkdir chess_autosaves && uv add pyproject.toml && uv run agent.py --game-id {game_id} --moves {NUM_MOVES}"

In [98]:
for clone, game_id, strategy in zip(clones, game_ids, strategies):
    print(f"Starting game {game_id} on instance {clone.id} with strategy {strategy}")
    run_agent_cmd = f"export OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY')} && cd app && mkdir chess_autosaves && uv add pyproject.toml && uv run agent.py --game-id {game_id} --moves 2"
    
    result = clone.exec(command=run_agent_cmd)
    print("="*50)
    print(f"Stdout:\n{result.stdout}")
    print("-"*50)
    print(f"Stderr:\n{result.stderr}")
    print('-'*50)
    # check that the game state saved
    result = clone.exec(command=f"cat app/chess_autosaves/game_id_{game_id}.json")
    print(f"Stdout:\n{result.stdout}")
    print("-"*50)
    print(f"Stderr:\n{result.stderr}")
    print("="*50)

Starting game 1 on instance morphvm_0lupadv4 with strategy aggressive
Stdout:
Using custom game ID: 1
Starting a new chess game...
Starting chess process with command: /root/app/.venv/bin/python3 ./chess_game.py --autosave --game-id 1 --strategy balanced
Chess process started, waiting for initial output...
Read line: New game started with ID: 1
Read line: 
Read line: Welcome to Chess!
Read line: You play as White, computer plays as Black
Read line: Agent is using the 'balanced' strategy
Read line: 
Read line: Commands:
Read line: - Enter moves in UCI format (e.g., 'e2e4')
Found prompt for move input!
Initial chess output received, parsing board state...
```json
{
  "selected_move": "e2e4",
  "alternative_moves": ["d2d4", "g1f3"]
}
```
Successfully parsed structured move: e2e4
LLM's move (balanced strategy): e2e4
Move 1 completed
```json
{
  "selected_move": "g1f3",
  "alternative_moves": ["d2d4", "f1c4"]
}
```
Successfully parsed structured move: g1f3
Failed to get a valid move from LL

In [108]:
print(clones[1].exec(f"cat app/chess_autosaves/game_id_2.json").stdout)

{
  "game_id": "2",
  "fen": "r3qb1r/pppkppp1/4bn2/3BP2p/1n1PN3/4BN2/PPPQ1PPP/R3K2R w KQ - 3 11",
  "move_history": [
    "e2e4",
    "d7d5",
    "g1f3",
    "b8c6",
    "b1c3",
    "h7h5",
    "d2d4",
    "c8f5",
    "f1c4",
    "c6b4",
    "c1e3",
    "e8d7",
    "d1d2",
    "d8c8",
    "e4e5",
    "c8e8",
    "c4d5",
    "f5e6",
    "c3e4",
    "g8f6"
  ],
  "move_timestamps": {
    "1": "2025-05-05T17:22:27.700322",
    "2": "2025-05-05T17:22:27.702911",
    "3": "2025-05-05T17:22:32.142952",
    "4": "2025-05-05T17:22:32.143787",
    "5": "2025-05-05T17:22:36.263046",
    "6": "2025-05-05T17:22:36.264687",
    "7": "2025-05-05T17:22:39.016279",
    "8": "2025-05-05T17:22:39.017144",
    "9": "2025-05-05T17:22:41.848616",
    "10": "2025-05-05T17:22:41.849968",
    "11": "2025-05-05T17:22:45.772518",
    "12": "2025-05-05T17:22:45.773354",
    "13": "2025-05-05T17:22:50.530099",
    "14": "2025-05-05T17:22:50.531022",
    "15": "2025-05-05T17:22:54.177509",
    "16": "2025-05-05T17

In [ ]:
print(clones[0].exec(f"ls app/chess_autosaves").stdout)


In [84]:
[x.id for x in clones]

['morphvm_0lupadv4', 'morphvm_pwkuoroc', 'morphvm_fa6uh8o9']

# utils

In [111]:
# delete all instances
instances = client.instances.list()

for instance in instances:
    print(f"Stopping instance ID: {instance.id}, Current Status: {instance.status}, Snapshot ID: {instance.refs.snapshot_id}")
    instance.stop()

Stopping instance ID: morphvm_fa6uh8o9, Current Status: ready, Snapshot ID: snapshot_sxocpfn8
Stopping instance ID: morphvm_0lupadv4, Current Status: ready, Snapshot ID: snapshot_sxocpfn8
Stopping instance ID: morphvm_pwkuoroc, Current Status: ready, Snapshot ID: snapshot_sxocpfn8
Stopping instance ID: morphvm_lxjfhhlo, Current Status: ready, Snapshot ID: snapshot_sxocpfn8


In [112]:
# delete all snapshots
snapshots = client.snapshots.list()

for snapshot in snapshots:
    print(f"Deleting snapshot ID: {snapshot.id}, Created At: {snapshot.created}")
    snapshot.delete()

Deleting snapshot ID: snapshot_sxocpfn8, Created At: 1746459331
Deleting snapshot ID: snapshot_j1nr3f6a, Created At: 1746459301
Deleting snapshot ID: snapshot_vh39mq3b, Created At: 1746459282
Deleting snapshot ID: snapshot_8sut7e5t, Created At: 1746459229
Deleting snapshot ID: snapshot_26bp6o23, Created At: 1746459191


In [109]:
# reset autosaves
for clone in clones:
    _ = clone.exec(command="rm -rf app/chess_autosaves")

In [110]:
# update agent.py
for clone in clones:
    with clone.ssh() as ssh:
        sftp = ssh._client.open_sftp()
        sftp.put("/home/asusevski/morph/chess/agent.py", "app/agent.py")